In [1]:
#from ckiptagger import data_utils, construct_dictionary

# 1.載入需要的package，定義處理資料的function "word_proc"

In [22]:
import tensorflow as tf
import keras

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc


In [70]:
def word_proc(data_path,
              column_name,
              tt_seg = (1,10),
              export_flag = False, 
              #tt_begin = 0,tt_end = 50,
              output_file = "df_test_out.csv"):
    """
        data_path(str):資料路徑(字串)，輸入的資料應為CSV檔，ex:
        column_name(str):要處理的欄位名稱
        tt_begin(int):起始列(整數)
        tt_end(int):結束列(整數)
        output_CSV_name:輸出的檔案名稱，指定為CSV檔    
    """
    from ckiptagger import WS, POS, NER
    
    file_type = data_path[-4:]
    print("輸入檔案格式為 {}".format(file_type))
    if 'csv' in file_type:
        data = pd.read_csv(data_path)
    elif 'xls' in file_type:
        data = pd.read_excel(data_path)
    else:
        #print("檔案須為CSV或XLS/XLSX")
        raise Exception("檔案須為CSV或XLS/XLSX")
        #return None
    
    column_list = list(data.columns)
    if column_name in column_list:
        text = data[column_name]
    else:
        print("輸入欄位\"{input_column}\"不存在\n匯入檔案欄位{i_column_name}".format(input_column = column_name,
                                                                       i_column_name=str(column_list)))
        raise Exception("不存在此欄位名稱")

    
    #ls_Test = list(text[tt_begin:tt_end])
    print("起始列數：{begin}\n結束列數:{end}".format(begin = (tt_seg[0]-1),end = tt_seg[1]))
    ls_Test = list(text[(tt_seg[0]-1):tt_seg[1]])
    
    
    print("Got list\n 載入 WS 資料...")
    ws = WS("./data")
    print("============================\n 開始執行 WS\n============================\n")
    word_sentence_list = ws(ls_Test
                                # sentence_segmentation=True, # To consider delimiters
                                # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}),
                                # This is the defualt set of delimiters
                                # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
                                # coerce_dictionary = dictionary2, # words in this dictionary are forced
                            )
    del ws
    del WS
    gc.collect()
    print("============================\n WS 完成\n============================\n載入 POS 資料...")
    
    print("============================\n 開始執行 POS \n============================\n")
    pos = POS("./data")
    pos_sentence_list = pos(word_sentence_list)
    del pos
    del POS
    gc.collect()
    print("============================\n POS 完成\n============================\n載入NER 資料...")
    
    
    print("============================\n 開始執行 NER \n============================\n")
    ner = NER("./data")
    entity_sentence_list = ner(word_sentence_list, pos_sentence_list)
    del NER
    del ner
    gc.collect()
    print("============================\n NER 完成\n============================\n")
        
    df_Test = pd.DataFrame(ls_Test,columns=[column_name])
    df_Test['WS'] = np.array(word_sentence_list)
    
    ##################
    #print("DEBUG")
    #print(pos_sentence_list[:5])
    ##################
    
    df_Test['POS'] = np.array(pos_sentence_list)
    df_Test['NER'] = np.array(entity_sentence_list)
    print("============================\n Dataframe Created \n============================\n")
    
    if export_flag is True:
        df_Test.to_csv(output_name)
        print("File Exported!!!!!")
    else:
        print("處理完成，不產生檔案")
        
    return df_Test,word_sentence_list,pos_sentence_list,entity_sentence_list

# 2.參數設定
### data_path_in    :要處理的檔案名稱(建議用絕對路徑)
### column_name_in  :要處理的欄位名稱
### proc_seg        :要處理的資料列起訖，格式為(起，迄)，均為整數，不可為0。
### export_flag     :是否要匯出檔案，True/False
### output_name     :匯出的檔案名稱，預設為CSV檔。

In [76]:
data_path_in = "KCC_Data/KCCNews11554.csv"
#data_path_c = 'KCC_Data/20160419-20190804-自由(985筆).xlsx'
column_name_in = "Text"
#column_name_c = "Teddd"
proc_seg = (1,10)
export_file = False
output_name = "mod_test.csv"

In [72]:
demo_df,ws_list,pos_list,ner_list = word_proc(data_path = data_path_in,
          column_name = column_name_in,
          tt_seg = proc_seg,export_flag = export_file ,output_file=output_name)

輸入檔案格式為 .csv
起始列數：0
結束列數:10
Got list
 載入 WS 資料...
 開始執行 WS

 WS 完成
載入 POS 資料...
 開始執行 POS 

 POS 完成
載入NER 資料...
 開始執行 NER 

 NER 完成

 Dataframe Created 

處理完成，不產生檔案


In [74]:
len(ner_list),type(ner_list[0]),len(ner_list[9])

(10, set, 45)

# 3.取出NER類別及專有名詞，分別存到df_class_list 及df_ner_list中
# 4.將其組成有兩個欄位['類別','專有名詞']的DataFrame。

In [77]:
df_class_list = list()
df_ner_list = list()

count = 0
for i in range(len(ner_list)):
    for j in range(len(ner_list[i])):
        count = count +1
        a = ner_list[i].pop()
        df_class_list.append(a[2])
        df_ner_list.append(a[3])
        #print(type(new_list[j]),(new_list[j]))
print(count)

352


In [78]:
extract_df = pd.DataFrame(
{'類別':df_class_list,
 '專有名詞':df_ner_list
})

# 5.檢視DataFrame資訊，查看各類別分別有多少資料

In [86]:
print(extract_df.describe())
extract_df.groupby('類別').count()

         類別 專有名詞
count   352  352
unique   15  199
top     GPE   高雄
freq    108   32


,專有名詞
類別,
CARDINAL,22
DATE,35
EVENT,3
FAC,12
GPE,108
LANGUAGE,1
LAW,1
LOC,1
MONEY,6


# 6-1.DataFrame處理 ─ 刪除重複資料

In [80]:
dd_extract_df = extract_df.drop_duplicates()
len(extract_df),len(dd_extract_df)

(352, 204)

# 6-2.DataFrame處理 ─ 挑出需要匯出的類別

In [88]:
dd_extract_df['類別'].unique()

array(['ORG', 'PERSON', 'GPE', 'FAC', 'DATE', 'TIME', 'LOC', 'ORDINAL',
       'PERCENT', 'MONEY', 'NORP', 'CARDINAL', 'LANGUAGE', 'LAW', 'EVENT'],
      dtype=object)

In [82]:
df_exp = dd_extract_df.loc[dd_extract_df['類別'].isin(['LOC','PERSON', 'ORG', 'LAW', 'EVENT','GPE'])]
len(df_exp)
#https://stackoverflow.com/questions/17071871/how-to-select-rows-from-a-dataframe-based-on-column-values

126

# 6-3.DataFrame處理 ─ 刪掉長度小於2的專有名詞

In [90]:
df_exp_2 = df_exp[df_exp['專有名詞'].map(len) >= 2].copy()
len(df_exp_2)

121

In [91]:
df_exp_2[:5].sort_values(by=['類別'])

,類別,專有名詞
4,GPE,臺灣
0,ORG,臺北國 家兩廳院
1,PERSON,馬英九
2,PERSON,簡文彬
3,PERSON,洪孟啟


# 7.將轉出的類別及專有名詞匯出成CSV檔
### (1)路徑：\\NER\\NER_export
### (2)檔案命名： NER_rlt_來源檔名_起始列數_結尾列數
### EX：來源檔名為KCCNews11554，本次處理的起訖列數為1~100
###     則檔名為 NER_rlt_KCCNews11554_1_100.csv


In [106]:
ex_NER_file_name =  "NER_rlt_KCCNews11553_1_10"
df_exp_2.to_csv("D:\python_kcc\\NER\\NER_export\\" + ex_NER_file_name + ".csv")